### Notebook to predict and save predictions and labels for accuracy metrics

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms
import torchvision
import torch

from PIL import Image
from torchvision.models.detection import FasterRCNN
from torch.utils.data import Dataset
from typing import Any

In [6]:
PROJECT_ROOT = 'C:/code/projects/python/applied-ai/'  # Enter local root dir

test_df = pd.read_csv(PROJECT_ROOT + 'data/obj_det/test_annotations.csv')
test_images = PROJECT_ROOT + 'data/obj_det/test'

test_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,ck0t4z1yrkmfv0794t2oqhd3f_jpeg.rf.17653d46c3ee...,640,480,smoke,514,209,616,285
1,ck0ujmglz85u10a468qt0d4fc_jpeg.rf.18d02755e8c9...,640,480,smoke,162,210,635,302
2,ck0uivtpc841h0a46ydgo7566_jpeg.rf.00134176dc29...,640,480,smoke,308,206,392,252
3,ck0nehpd69bax0721onacbe33_jpeg.rf.02ed50fbcb97...,640,480,smoke,274,229,454,311
4,ck0ow7vs07tuz08485n4yz5si_jpeg.rf.179fc0e59422...,640,480,smoke,386,218,607,285


In [7]:
LABELS = test_df['class'].unique()
NUM_OF_CLASSES = len(LABELS)+1
SAVE_PATH = '../models/rcnn/'
MODEL_NAME = 'model_new.pt'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TRANSFORM = transforms.Compose([transforms.ToTensor()])

In [9]:
class LabelMap:
    def __init__(self, labels: list) -> None:
        self._map = {c: i+1 for i, c in enumerate(labels)}
        self.reversed_map = {i: c for i, c in enumerate(labels)}

    def fit(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
        df[col] = df[col].map(self._map)
        return df


class WildfireDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_path: str, labels: list, transforms: Any = None, **kwargs) -> None:
        super().__init__(**kwargs)
        self.df = df
        self.img_path = img_path
        self.labels = labels
        self.images = self.df['filename'].unique()
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, i: int) -> tuple:
        img_file = os.path.join(self.img_path, self.images[i])

        img = Image.open(img_file)

        img_data = self.df.loc[self.df['filename'] == self.images[i]]

        xmins = img_data['xmin'].values
        ymins = img_data['ymin'].values
        xmaxs = img_data['xmax'].values
        ymaxs = img_data['ymax'].values

        boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
        labels = torch.as_tensor(img_data['class'].values, dtype=torch.int64)
        _id = torch.tensor([i])

        areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
        areas = torch.as_tensor(areas, dtype=torch.float32)

        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        target = dict()
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = _id
        target['area'] = areas
        target['iscrowd'] = iscrowd

        if self.transforms:
            img = self.transforms(img)
    
        return torch.as_tensor(img, dtype=torch.float32), target

    def get_h_w(self, image: str) -> tuple:
        """Get height and width of image"""
        img_data = self.df.loc[self.df['filename'] == image]
        return img_data['width'].values[0], img_data['height'].values[0]

In [10]:
def encode_label(df: pd.DataFrame, col: str, map_dict: dict) -> pd.DataFrame:
    df[col] = df[col].map(map_dict)
    return df

def collate_fn(batch: tuple) -> tuple:
    return tuple(zip(*batch))

def load_model(model_path: str) -> FasterRCNN:
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=False,
        num_classes=NUM_OF_CLASSES
    )
    IN_FEATURES = model.roi_heads.box_predictor.cls_score.in_features

    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def predict(model, img: Image) -> tuple:
    pred_img = img.view(1, 3, img.shape[1], img.shape[2])

    preds = model(pred_img)
    outputs = [{k: v.to(torch.device('cpu')) for k, v in target.items()} for target in preds]

    boxes = outputs[0]['boxes'].data.cpu().numpy().astype(np.int32)
    scores = outputs[0]['scores'].data.cpu().numpy()
    labels = outputs[0]['labels'].data.cpu().numpy().astype(np.int32)
    
    return boxes, scores, labels

In [11]:
model = load_model(SAVE_PATH + MODEL_NAME)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [12]:
label_map = LabelMap(LABELS)
test_df = encode_label(test_df, 'class', label_map._map)

In [13]:
test_dataset = WildfireDataset(test_df, test_images, LABELS, TRANSFORM)

In [15]:
MAP_targets, MAP_preds = list(), list()

for image, target in test_dataset:
    prediction = predict(model, image)
    MAP_preds.append(dict(
        boxes=torch.tensor(prediction[0]),
        scores=torch.tensor(prediction[1]),
        labels=torch.tensor(prediction[2])
    ))
    MAP_targets.append(dict(
        boxes=target['boxes'],
        labels=target['labels']
    ))

MAP_preds[2]

{'boxes': tensor([[302, 204, 412, 252]], dtype=torch.int32),
 'scores': tensor([0.9433]),
 'labels': tensor([1], dtype=torch.int32)}

In [ ]:
import pickle

with open(PROJECT_ROOT + 'data/pickles/map_pred.pkl', 'wb') as f:
    pickle.dump(MAP_preds, f)

with open(PROJECT_ROOT + 'data/pickles/map_target.pkl', 'wb') as f:
    pickle.dump(MAP_targets, f)